## 1. Libraries

In [4]:
import os
import numpy as np
import pandas as pd

#Janome
from janome.tokenizer import Tokenizer
from janome.analyzer import Analyzer
from janome.charfilter import *
from janome.tokenfilter import *

#Mecab
import MeCab

from bs4 import BeautifulSoup
import requests
import re, regex
import collections

from gensim.models import Word2Vec
import codecs
from tqdm import tqdm

/opt/anaconda3/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


## 2. Function

In [5]:
def get_text(url):
    # input(str): Word2Vecの解析に利用するデータになる
    # output(list of list of str): 段落ごとにリストを作成し、全段落を1つの2次元リストとして出力する
    # html.parserでURLの<p>タグコンテンツのみを抽出する。
    
    list_of_paragraphs = []
    
    r = requests.get(url)
    c = r.content
    
    # サイトの重要なコンテンツのみを抽出
    soup = BeautifulSoup(c,"html.parser")
    content = soup.find_all("p")
    
    # 段落ごとにリストにアペンド
    for para in content:
        paragraph = para.text
        list_of_paragraphs.append(paragraph)

    return list_of_paragraphs

def remove_excess(text):
    # input(str): Raw文章
    # output(str): 不要な記号やURLを削除した文章
    # Strに含む記号やURLを削除する
    
    # URL削除
    result = re.sub(r'https?://[\w/:%#\$&\?\(\)~\.=\+\-]+', '', str(text))
    
    # 記号削除
    code_regex = regex.compile(r'[1234567890１２３４５６７８９０¥t¥s!"#$%&¥¥¥¥¥()*+,-./:;；：<=>?@[¥¥]^_`{|}~○「」「」〔〕“”〈〉''『』【】＆＊（）＄＃＠？！｀＋￥¥％♪...◇→←↓↑。・ω・。 ゚ ́∀｀ΣДｘ①②③④⑤⑥⑦⑧⑨◎©︎♡★☆▽※ゞノ〆εσ＞＜┌┘]')
    result = code_regex.sub('', result)
                               
    return result

def remove_stopwords(all_tokens): #FAST #input: list of all tokens
    # input(list of str): Tokenizeされたリスト（JanomeやMeCabで文章をTokenizeしリスト化した後）
    # output(list of str): ストップワードを除いたリスト
    # stop_words.txtに記載されているストップワードがリストの単語と一致する場合削除する
    
#     stop_words_ja=[]
                               
#     file = codecs.open("stop_words.txt", "r",'utf_8')
#     lines = file.readlines()
#     file.close()
#     for w in lines:
#         stop_words_ja.append(w.replace('¥n', ''))
    s = "。 は 、 の （ ） に で を た し が と )', て ある れ ', さ する いる ． から ' も ・ として . 「 」 い こと – な なっ や れる など ため この まで また あっ ない あり なる その られ 後 『 』 へ 日本 という よう ( 現在 もの より だ おり 的 中 により ) 2 による 第 なり によって 1 これ その後 ず , か 時 なく られる だっ において 者 なかっ 行わ 多く しかし 3 せ 他 名 出身 それ について 間 当時 上 ば 存在 受け . 呼ば 同 なお できる 目 行っ 内 う 数 のみ 前 以下 き ： 元 化 4 等 および 使用 でき 同年 主 場合 際 一 約 における さらに 一部 所属 人 以降 ら 活動 5 中心 作品 いう 知ら 同じ 初 だけ 多い 時代 以上 生まれ 発表 2010年 にて 見 務め 持つ とともに 大 参加 頃 位置 2007年 2009年 2008年 開始 うち 行う ほか 特に 全 ながら 当初 発売 せる 2011年 家 かつて 下 卒業 一つ 2006年 6 でも 年 2012年 形 用い に対して 最初 / 本 考え なら 以外 関係 一方 それぞれ 各 同様 4月 経 2013年 と共に 2005年 そして 3月 地域 必要 これら 及び 一般 用 2014年 結果 可能 現 開催 事 ものの 利用 にかけて 部 影響 設立 記録 得 アメリカ 通り とも 彼 2015年 自身 登場 始め または 担当 変更 意 味 たり 側 とき 開発 設置 代表 ほど ので 構成 ただし 二 2004年 郡 初めて たち 部分 2016年 最も 放送 7 旧 地 最後 アメリカ合衆国 10月 世界 研究 大学 8 系 大きな 活躍 獲得 続け 以前 全て 問題 性 与え 9月 父 含む といった ほとんど 7月 ところ 2017年 2003年 向け 持っ 2000年 加え 使わ 型 6月 に関する 出場 12月 目的 高い 名称 に対する 1月 万 実際 5月 -1 名前 様々 再び 10"
    stop_words_ja = s.split(" ")
                               
    clean_tokens = [x for x in all_tokens if x not in set(stop_words_ja)]

    return clean_tokens
                               
                    
def tokenize2_janome(list_of_paragraphs): #FASTER
    # input(list of list of str): 元データファイル(csv)の各行をリスト化した2次元リスト
    # output(list of list of str): Tokenizeされたリスト
    # csvファイルの各行の文章をJanomeのAnalyzerでTokenizeしている。char_filterで文字化けを修正し、token_filterで複合名詞を感知する。
    # remove_excessで不要な記号を削除する。remove_stopwordsでストップワードを削除する
    # Janomeを利用したい場合はこのファンクションを使い、tokenize2_mecabは使わない
    
    char_filters = [UnicodeNormalizeCharFilter(), RegexReplaceCharFilter('<.*?>', '')]

    token_filters = [CompoundNounFilter(), POSKeepFilter(['名詞']), LowerCaseFilter(), ExtractAttributeFilter('surface')]
    
    a = Analyzer(char_filters = char_filters,
                 token_filters = token_filters)
                               
    list_for_w2v = []
    for x in list_of_paragraphs:
        temp = []
        text = remove_excess(x) # FAST

        for token in a.analyze(text):
            temp.append(token)
                    
        clean_tokens = remove_stopwords(temp) # FAST

        list_for_w2v.append(clean_tokens)
                               
    return list_for_w2v

def tokenize2_mecab(list_of_paragraphs):
    # input(list of list of str): 元データファイル(csv)の各行をリスト化した2次元リスト
    # output(list of list of str): Tokenizeされたリスト
    # csvファイルの各行の文章をMeCabのTaggerとNeologdでTokenizeしている。Neologdで複合名詞を感知する。Parse.splitlinesで名詞のみを抽出
    # remove_excessで不要な記号を削除する。remove_stopwordsでストップワードを削除する
    # MeCabを利用したい場合はこのファンクションを使い、tokenize2_janomeは使わない
    
    m = MeCab.Tagger("-Ochasen -d /usr/local/lib/mecab/dic/mecab-ipadic-neologd")

    list_for_w2v = []
    for x in list_of_paragraphs:
        temp = []
        text = remove_excess(x) # FAST        
        pos_words = [text for text in m.parse(text).splitlines()
                    if "名詞" in text.split()[-1]]
        for string in pos_words:
            t = string.split()[0]
            temp.append(t)
        clean_tokens = remove_stopwords(temp) # FAST
        list_for_w2v.append(clean_tokens)

    return list_for_w2v

def topN_dougigo(token_2d_list, topN, threshold, model):
    # input(token_2d_list(list of list of str): Tokenizeされたリスト
    #       TopN(int):ユーザーが観覧したい上位単語数
    #       threshold(float):ユーザーが観覧したい同義語の類似度の閾値     
    #       model(Word2Vec model):事前に学習されたmodel         
    # output(Pandas DataFrame): 出現頻度上位の同義語DataFrame
    # Tokenリストから出現頻度順に単語を並べ、各単語に対する同義語を出力する
    # 単語ごとに出現頻度が異なるため、ユーザーに上位〇〇件（TopN）を指定してもらう
    # 単語ごとの同義語の類似度は異なるため、ユーザーに類似度の閾値（threshold）を指定してもらい、閾値を超える単語のみ出力する
    # modelはpv_data.csvで事前に学習させたデータを利用することをおすすめする
    
    print("２次元リストを１次元化中...")
    token_1d_list = [item for sublist in tqdm(token_2d_list) for item in sublist]
    counter=collections.Counter(token_1d_list)
    top_list = counter.most_common(topN) # [("処方", 45000), ("患者", 35000), ...]
    
    columns = ["単語", "出現回数", "同義語", "類似度"]
    
    df = pd.DataFrame(columns = columns)
    
    print("出現頻度上位", topN, "件の同義語DataFrameを作成中...")
    for (top_word, top_freq) in tqdm(top_list): # loops topN times
        ms_list  =  model.wv.most_similar(top_word)  #  Top  10  most  similar  of  "処方" →[("処方箋",  0.9505),("処方薬", 0.9435),...]
        for (ms_word, similarity) in ms_list: # loops topN x 10 times
            if similarity >= threshold:
                temp = pd.DataFrame([[top_word, top_freq, ms_word, similarity]], columns=columns)
                df = df.append(temp)
    
    print("※閾値（", threshold, "）を下回る同義語に関しては非表示にしています。")
    
    return df

def ask_topN():
    # input(int): ユーザーが観覧したい上位単語数
    # output(int): ユーザーが観覧したい上位単語数
    # ユーザーに直接入力させる。intではない場合繰り返して聞く
    
    topN = input("単語出現頻度上位何件分を検索しますか？：")
    
    try:
        topN = int(topN)
        print("> 上位", topN, "件に設定。")
    except ValueError:
        print("数字を入力してください。")
        ask_topN()
    
    return topN

def ask_threshold():
    # input(int): ユーザーが観覧したい同義語の類似度の閾値
    # output(int): ユーザーが観覧したい同義語の類似度の閾値
    # ユーザーに直接入力させる。0-1のfloatではない場合繰り返して聞く
    
    while True:
        threshold = float(input("同義語としてピックアップするための類似度（0～1）を設定してください："))
        if threshold < 1 and threshold >= 0:
            break
        print("0-1の数字を入力してください。")
    
    print("> 類似度の閾値を", threshold, "に設定。")
    
    return threshold

## 3. Import Data

In [24]:
url = "https://medical.nikkeibp.co.jp/inc/all/drugdic/prd/39/3969010F2022.html"
url2 = "https://medical.nikkeibp.co.jp/inc/all/drugdic/prd/39/3969019F2023.html"
url3 = "https://www.kegg.jp/medicus-bin/japic_med?japic_code=00061994"
url4 = "https://medical.nikkeibp.co.jp/inc/all/drugdic/prd/39/3969010F2022.html"
url5 = "http://www.interq.or.jp/ox/dwm/se/se11/se1141007.html"
url6 = "http://www.okusuri110.com/cgi-bin/biyokibetu_disp.cgi?02&02-08"
url7 = "http://www.okusuri110.com/cgi-bin/biyokibetu_disp.cgi?13&13-12"
url8 = "http://www.okusuri110.jp/pc/yaka/yaka2021/yaka_index_new2021.html#5"
url9 = "https://ja.wikipedia.org/wiki/%E5%8C%BB%E8%96%AC%E5%93%81"
url10 = "https://ja.wikipedia.org/wiki/%E7%B3%96%E5%B0%BF%E7%97%85"
url11 = "https://dm-net.co.jp/calendar/2019/029184.php"
url12 = "https://ja.wikipedia.org/wiki/%E7%97%85%E6%B0%97"
url13 = 'https://ja.wikipedia.org/wiki/%E7%97%85%E9%99%A2'
url14 = 'https://ja.wikipedia.org/wiki/%E5%8C%BB%E5%B8%AB'
url15 = "https://ja.wikipedia.org/wiki/%E5%8C%BB%E7%99%82"
url16 = 'https://dm-rg.net/contents/sglt2_introduction'
url17 = 'https://dm-rg.net/guide/DPP4_inhibitor_list'
url18 = 'https://www.ononavi1717.jp/area/diabetes/forxiga/drug-faq/40621'
url19 = 'https://www.ono.co.jp/news/20210826.html'

df_list = []

for i in [url, url2, url3, url4, url5, url6, url7, url8, url9, url10, url11, url12, url13, url14, url15, url16, url17, url18, url19]:
    temp = get_text(i)
    df_list.append(temp)

## 4. Tokenize

In [25]:
token_FULL_janome = tokenize2_janome(df_list)

In [26]:
token_FULL_mecab = tokenize2_mecab(df_list)

## 5. Create Model

In [27]:
model_FULL_janome = Word2Vec(token_FULL_janome, min_count = 1, vector_size = 300, seed = 42)

In [28]:
model_FULL_mecab = Word2Vec(token_FULL_mecab, min_count = 1, vector_size = 300, seed = 42)

## 6. Automatically Generate Words - User Interface

In [29]:
topN = ask_topN()
threshold = ask_threshold()

単語出現頻度上位何件分を検索しますか？：30
> 上位 30 件に設定。
同義語としてピックアップするための類似度（0～1）を設定してください：0.7
> 類似度の閾値を 0.7 に設定。


In [30]:
temp = topN_dougigo(token_FULL_janome, topN, threshold, model_FULL_janome)
temp.head(30)

100%|██████████| 30/30 [00:00<00:00, 1647.62it/s]

２次元リストを１次元化中...
出現頻度上位 30 件の同義語DataFrameを作成中...
※閾値（ 0.7 ）を下回る同義語に関しては非表示にしています。


,単語,出現回数,同義語,類似度


In [31]:
temp2 = topN_dougigo(token_FULL_mecab, topN, threshold, model_FULL_mecab)
temp2.head(30)

 17%|█▋        | 5/30 [00:00<00:00, 42.52it/s]

２次元リストを１次元化中...
出現頻度上位 30 件の同義語DataFrameを作成中...


100%|██████████| 30/30 [00:00<00:00, 50.29it/s]

※閾値（ 0.7 ）を下回る同義語に関しては非表示にしています。


,単語,出現回数,同義語,類似度
0,n,344,糖尿病,0.969212
0,n,344,医薬品,0.968015
0,n,344,病気,0.964907
0,n,344,患者,0.962442
0,n,344,症状,0.956748
0,n,344,疾患,0.955159
0,n,344,状態,0.954260
0,n,344,医師,0.946582
0,n,344,投与,0.944976
0,n,344,医療,0.944839


In [400]:
import pickle

filename = "token_FULL_pickle"
outfile = open(filename, "wb")

In [402]:
pickle.dump(token_FULL_janome, outfile)

In [404]:
outfile.close()

In [405]:
infile = open(filename,'rb')
new_dict = pickle.load(infile)
infile.close()

In [407]:
print(new_dict==token_FULL_janome)

True


In [410]:
topN = ask_topN()
threshold = ask_threshold()

単語出現頻度上位何件分を検索しますか？：100
> 上位 100 件に設定。
同義語としてピックアップするための類似度（0～1）を設定してください：0
> 類似度の閾値を 0.0 に設定。


In [413]:
temp2 = topN_dougigo(new_dict, topN, threshold, model_FULL_janome)
temp2.head(30)

  5%|▌         | 5/100 [00:00<00:02, 44.21it/s]

２次元リストを１次元化中...
出現頻度上位 100 件の同義語DataFrameを作成中...


100%|██████████| 100/100 [00:01<00:00, 50.32it/s]

※閾値（ 0.0 ）を下回る同義語に関しては非表示にしています。


,単語,出現回数,同義語,類似度
0,"\n',",169,医師,0.327541
0,"\n',",169,患者,0.286433
0,"\n',",169,病院,0.273250
0,"\n',",169,医療,0.261470
0,"\n',",169,病気,0.261030
0,"\n',",169,2型糖尿病,0.248214
0,"\n',",169,効果,0.233409
0,"\n',",169,糖尿病,0.233188
0,"\n',",169,1型糖尿病,0.229944
0,"\n',",169,治療,0.229732


In [414]:
temp2 = topN_dougigo(token_FULL_janome, topN, threshold, model_FULL_janome)
temp2.head(30)

  6%|▌         | 6/100 [00:00<00:01, 48.67it/s]

２次元リストを１次元化中...
出現頻度上位 100 件の同義語DataFrameを作成中...


100%|██████████| 100/100 [00:02<00:00, 49.03it/s]

※閾値（ 0.0 ）を下回る同義語に関しては非表示にしています。


,単語,出現回数,同義語,類似度
0,"\n',",169,医師,0.327541
0,"\n',",169,患者,0.286433
0,"\n',",169,病院,0.273250
0,"\n',",169,医療,0.261470
0,"\n',",169,病気,0.261030
0,"\n',",169,2型糖尿病,0.248214
0,"\n',",169,効果,0.233409
0,"\n',",169,糖尿病,0.233188
0,"\n',",169,1型糖尿病,0.229944
0,"\n',",169,治療,0.229732
